In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import glob
import pandas as pd
import os
import numpy as np
import pickle
import sys

In [3]:
import dataset_confs

In [4]:
from DatasetManager import DatasetManager

/gpfs/hpchome/etais/hpc_irheta/.conda/envs/bpm3/lib/python3.5/site-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using Theano backend.


# Dataset stats

In [ ]:
home_dir = ""

In [ ]:
dt_stats = []
datasets = ["bpic2011_f%s"%formula for formula in range(1,5)] + ["bpic2015_%s_f2"%(municipality) for municipality in range(1,6)] + ["production",
                                                                                                                                    "insurance_activity", "insurance_followup",
                                                                                                                                    "sepsis_cases_1", "sepsis_cases_2", "sepsis_cases_4",
                                                                                                                                    "bpic2012_accepted", "bpic2012_declined", "bpic2012_cancelled",
                                                                                                                                    "bpic2017_accepted", "bpic2017_refused", "bpic2017_cancelled",
                                                                                                                                    "traffic_fines_1", "hospital_billing_2", "hospital_billing_3"]
datasets = ["unemployment"]
print("\\toprule  &  & min  & med  & max  & trunc  & \\# variants & pos class  & \\# event & \\# static  & \\# dynamic  & \\# static  & \\# dynamic \\\\ ")
print(" dataset & \\# traces &  length &  length &  length &  length & (after trunc) & ratio & classes &  attr-s & attr-s &  cat levels & cat levels\\\\ \\midrule")
for dataset_name in datasets:
    dataset_manager = DatasetManager(dataset_name)
    
    case_id_col = dataset_confs.case_id_col[dataset_name]
    activity_col = dataset_confs.activity_col[dataset_name]
    timestamp_col = dataset_confs.timestamp_col[dataset_name]
    label_col = dataset_confs.label_col[dataset_name]
    pos_label = dataset_confs.pos_label[dataset_name]

    dynamic_cat_cols = dataset_confs.dynamic_cat_cols[dataset_name]
    static_cat_cols = dataset_confs.static_cat_cols[dataset_name]
    dynamic_num_cols = dataset_confs.dynamic_num_cols[dataset_name]
    static_num_cols = dataset_confs.static_num_cols[dataset_name]

    data_filepath = os.path.join(home_dir, dataset_confs.filename[dataset_name])

    # specify data types
    dtypes = {col:"object" for col in dynamic_cat_cols+static_cat_cols+[case_id_col, label_col, timestamp_col]}
    for col in dynamic_num_cols + static_num_cols:
        dtypes[col] = "float"
            
    data = pd.read_csv(data_filepath, sep=";", dtype=dtypes)
    sizes = data.groupby(case_id_col).size()

    class_freqs = data.groupby(case_id_col).first()[label_col].value_counts()
    
    if "traffic_fines" in dataset_name:
        max_prefix_length = 10
    elif "bpic2017" in dataset_name:
        max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
    else:
        max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))
    
    n_trace_variants = len(data.sort_values(timestamp_col, kind="mergesort").groupby(case_id_col).head(max_prefix_length).groupby(case_id_col)[activity_col].apply(lambda x: "__".join(list(x))).unique())
    
    n_static_cat_levels = 0
    n_dynamic_cat_levels = 0
    for col in dynamic_cat_cols:
        n_dynamic_cat_levels += len(data[col].unique())
    for col in static_cat_cols:
        n_static_cat_levels += len(data[col].unique())
    
    dataset_name = dataset_name.replace("_", "\\_").replace("bpic2011\_f", "bpic2011\_").replace("\_f2", "").replace("activity", "1").replace("followup", "2").replace("cases\_", "").replace("billing\_", "").replace("accepted", "1").replace("declined", "2").replace("refused", "2").replace("cancelled", "3").replace("\_fines\_1", "").replace("sepsis\_4", "sepsis\_3").replace("hospital\_2", "hospital\_1").replace("hospital\_3", "hospital\_2")
    print("%s & %s & %s & %s & %s & %s & %s & %s & %s & %s & %s & %s & %s \\\\"%(dataset_name, len(data[case_id_col].unique()), sizes.min(), sizes.quantile(0.50), sizes.max(), max_prefix_length, n_trace_variants, round(class_freqs[pos_label] / len(data[case_id_col].unique()), 2), len(data[activity_col].unique()), len(static_cat_cols) + len(static_num_cols), len(dynamic_cat_cols) + len(dynamic_num_cols),
                                                                      n_static_cat_levels, n_dynamic_cat_levels))
    
    record = (dataset_name, len(data[case_id_col].unique()), sizes.min(), sizes.quantile(0.50), sizes.max(), max_prefix_length, n_trace_variants, round(class_freqs[pos_label] / len(data[case_id_col].unique()), 2), len(data[activity_col].unique()), len(static_cat_cols) + len(static_num_cols), len(dynamic_cat_cols) + len(dynamic_num_cols),
                                                                      n_static_cat_levels, n_dynamic_cat_levels)
    dt_stats.append(record)
    
print("\\bottomrule")

# Execution times

In [143]:
data = pd.DataFrame()
for file in glob.glob("results_performance/*"):
    tmp = pd.read_csv(file, sep=";")
    
    if "single_agg.csv" in file:
        tmp["bucket_enc"] = "single_agg"
        
    elif "single_laststate.csv" in file:
        tmp["bucket_enc"] = "single_laststate"
        
    elif "prefix_index.csv" in file:
        tmp["bucket_enc"] = "prefix_index_last"
        data = pd.concat([data, tmp], axis=0)
        tmp["bucket_enc"] = "prefix_index_agg"
        
    data = pd.concat([data, tmp], axis=0)

In [144]:
data2 = pd.DataFrame()
for file in glob.glob("results_performance2/*"):
    tmp = pd.read_csv(file, sep=";")
    
    if "single_agg.csv" in file:
        tmp["bucket_enc"] = "single_agg"
        
    elif "single_laststate.csv" in file:
        tmp["bucket_enc"] = "single_laststate"
        
    elif "prefix_index.csv" in file:
        tmp["bucket_enc"] = "prefix_index_last"
        data2 = pd.concat([data2, tmp], axis=0)
        tmp["bucket_enc"] = "prefix_index_agg"
        
    data2 = pd.concat([data2, tmp], axis=0)

In [146]:
data2 = data2[~pd.isnull(data2.score)]
data1 = data2.merge(data, on=["dataset", "bucket_enc", "text_method_enc", "cls", "metric"])
data1 = data1[data1.metric=="offline_total_avg"]
data1["offline_total_avg"] = data1[["score_x", "score_y"]].mean(axis=1)
data1["offline_total_std"] = data1[["score_x", "score_y"]].std(axis=1)
data1 = data1.drop(["metric", "score_x", "score_y"], axis=1)
data1 = pd.melt(data1, id_vars=["dataset", "bucket_enc", "text_method_enc", "cls"], value_vars=['offline_total_avg', 'offline_total_std'])
data1.columns = ["dataset", "bucket_enc", "text_method_enc", "cls", "metric", "score"]
data1.set_index(["dataset", "bucket_enc", "text_method_enc", "cls", "metric"], inplace=True)
data.set_index(["dataset", "bucket_enc", "text_method_enc", "cls", "metric"], inplace=True)
data.update(data1)
data.reset_index(inplace=True)

In [165]:
data.head()

,dataset,bucket_enc,text_method_enc,cls,metric,score
0,dc,single_laststate,no_text,xgboost,offline_total_avg,64.302090
1,dc,single_laststate,no_text,xgboost,offline_total_std,0.274182
2,dc,single_laststate,no_text,xgboost,online_event_avg,0.022199
3,dc,single_laststate,no_text,xgboost,online_event_std,0.007037
4,dc,single_laststate,no_text,logit,offline_total_avg,5.506207


In [166]:
data.dataset = data.dataset.str.replace("dc", "DR")
data.dataset = data.dataset.str.replace("crm2", "LtC")
data.dataset = data.dataset.str.replace("github", "Github")

data.bucket_enc[data.text_method_enc.str.contains("laststate")] = "prefix_index_last"
data.bucket_enc[data.text_method_enc.str.contains("agg")] = "prefix_index_agg"

data.text_method_enc = data.text_method_enc.str.replace("_laststate", "")
data.text_method_enc = data.text_method_enc.str.replace("_agg", "")

data.text_method_enc = data.text_method_enc.str.replace("no_text", "no text")
data.text_method_enc = data.text_method_enc.str.replace("nb", "NB")
data.text_method_enc = data.text_method_enc.str.replace("pv", "PV")
data.text_method_enc = data.text_method_enc.str.replace("lda", "LDA")
data.text_method_enc = data.text_method_enc.str.replace("bong", "BoNG")

/gpfs/hpchome/etais/hpc_irheta/.conda/envs/bpm3/lib/python3.5/site-packages/ipykernel/__main__.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/gpfs/hpchome/etais/hpc_irheta/.conda/envs/bpm3/lib/python3.5/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [167]:
data.tail()

,dataset,bucket_enc,text_method_enc,cls,metric,score
967,LtC,prefix_index_agg,PV,logit,offline_total_std,3765.835587
968,LtC,prefix_index_agg,PV,logit,online_event_avg,0.156071
969,LtC,prefix_index_agg,PV,logit,online_event_std,0.083138
970,LtC,prefix_index_agg,PV,logit,offline_text_model_avg,268.314868
971,LtC,prefix_index_agg,PV,logit,offline_text_model_std,0.000000


In [179]:
cls = "rf"
data_cls = data[data.cls==cls]

In [180]:
best_offline_times = data_cls[data_cls.metric=="offline_total_avg"].groupby(["dataset"])["score"].min().round(2)
best_online_times = data_cls[data_cls.metric=="online_event_avg"].groupby(["dataset"])["score"].min().round(2)

In [181]:
bucket_encs = ["single_agg", "single_laststate", "prefix_index_agg", "prefix_index_last"]
text_methods = ["no text", "BoNG", "NB", "LDA", "PV"]
datasets = ["DR", "LtC", "Github"]

In [182]:
print("\\toprule")
for bucket_enc in bucket_encs:
    print("%s \\\\ \\cmidrule(lr){2-3} \\cmidrule(lr){4-5} \\cmidrule(lr){6-7}"%("".join([" & \multicolumn{2}{c}{{\\bfseries %s}}"%(dataset.replace("_", "\\_")) for dataset in datasets])))
    print(bucket_enc.replace("_", "\\_") + " & offline\\_total (s) & online\\_avg (ms)" * len(datasets) + " \\\\ \midrule")
    for method_name in text_methods:
        elems_to_print = [method_name.replace("_", "\\_")]
        for dataset in datasets:
            
            dt_sub = data_cls[(data_cls.dataset==dataset) & (data_cls.bucket_enc==bucket_enc) & (data_cls.text_method_enc==method_name)]
            if len(dt_sub) > 0:
                offline_mean = round(dt_sub[dt_sub.metric=="offline_total_avg"].score.iloc[0], 2)
                offline_std = round(dt_sub[dt_sub.metric=="offline_total_std"].score.iloc[0], 2)
                online_mean = round(dt_sub[dt_sub.metric=="online_event_avg"].score.iloc[0], 2)
                online_std = round(dt_sub[dt_sub.metric=="online_event_std"].score.iloc[0], 2)
                
                if offline_mean == best_offline_times[dataset]:
                    elems_to_print.append("$\\mathbf{%s \\pm %s}$" % (offline_mean, offline_std))
                else:
                    elems_to_print.append("$%s \\pm %s$" % (offline_mean, offline_std))

                if online_mean == best_online_times[dataset]:
                    elems_to_print.append("$\\mathbf{%s \\pm %s}$" % (online_mean, online_std))
                else:
                    elems_to_print.append("$%s \\pm %s$" % (online_mean, online_std))
                
            else:
                elems_to_print.extend(["-", "-"])

        print("%s \\\\ "%(" & ".join(elems_to_print)))
    print("\\bottomrule")

\toprule
 & \multicolumn{2}{c}{{\bfseries DR}} & \multicolumn{2}{c}{{\bfseries LtC}} & \multicolumn{2}{c}{{\bfseries Github}} \\ \cmidrule(lr){2-3} \cmidrule(lr){4-5} \cmidrule(lr){6-7}
single\_agg & offline\_total (s) & online\_avg (ms) & offline\_total (s) & online\_avg (ms) & offline\_total (s) & online\_avg (ms) \\ \midrule
no text & $19.83 \pm 0.58$ & $0.21 \pm 0.07$ & $1648.58 \pm 82.5$ & $0.09 \pm 0.11$ & $1043.73 \pm 40.63$ & $\mathbf{0.07 \pm 0.05}$ \\ 
BoNG & $29.66 \pm 0.84$ & $0.37 \pm 0.12$ & $22761.75 \pm 456.53$ & $0.11 \pm 0.13$ & $2286.96 \pm 241.26$ & $0.56 \pm 0.34$ \\ 
NB & $81.03 \pm 1.46$ & $0.23 \pm 0.07$ & $18208.0 \pm 0.0$ & $0.11 \pm 0.13$ & $2456.88 \pm 189.07$ & $0.14 \pm 0.09$ \\ 
LDA & $134.16 \pm 2.14$ & $0.22 \pm 0.07$ & $10303.79 \pm 482.55$ & $\mathbf{0.05 \pm 0.06}$ & $2325.06 \pm 21.31$ & $0.14 \pm 0.08$ \\ 
PV & $34.4 \pm 8.0$ & $0.25 \pm 0.27$ & $3855.07 \pm 174.3$ & $0.11 \pm 0.79$ & $945.45 \pm 17.32$ & $0.21 \pm 0.24$ \\ 
\bottomrule
 & \multico

# Overall AUC

In [18]:
data = pd.DataFrame()
for file in glob.glob("results_unstructured/*"):
    tmp = pd.read_csv(file, sep=";")
    if not("bong" in file or "pv" in file or "lda" in file or "nb" in file):
        tmp.rename(index=str, columns={"method": "bucket_enc"}, inplace=True)
        tmp["text_method_enc"] = "no text"
    data = pd.concat([data, tmp], axis=0)

/gpfs/hpchome/etais/hpc_irheta/.conda/envs/bpm3/lib/python3.5/site-packages/ipykernel/__main__.py:7: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



In [19]:
with open("n_test_cases.pickle", "rb") as fin:
    test_cases_dict = pickle.load(fin)

In [20]:
df_test_cases = pd.DataFrame(test_cases_dict)
df_test_cases = df_test_cases.stack().reset_index()
df_test_cases.columns = ["nr_events", "dataset", "n_cases"]

In [ ]:
data = pd.merge(data, df_test_cases, on=["dataset", "nr_events"])

In [ ]:
data.dataset = data.dataset.str.replace("dc", "DR")
data.dataset = data.dataset.str.replace("crm2", "LtC")
data.dataset = data.dataset.str.replace("github", "Github")

data.bucket_enc[data.bucket_enc=="agg"] = "single_agg"
data.bucket_enc[data.text_method_enc.str.contains("laststate")] = "prefix_index_last"
data.bucket_enc[data.text_method_enc.str.contains("agg")] = "prefix_index_agg"

data.text_method_enc = data.text_method_enc.str.replace("_laststate", "")
data.text_method_enc = data.text_method_enc.str.replace("_agg", "")

data.text_method_enc = data.text_method_enc.str.replace("no_text", "no text")
data.text_method_enc = data.text_method_enc.str.replace("nb", "NB")
data.text_method_enc = data.text_method_enc.str.replace("pv", "PV")
data.text_method_enc = data.text_method_enc.str.replace("lda", "LDA")
data.text_method_enc = data.text_method_enc.str.replace("bong", "BoNG")

In [ ]:
data.bucket_enc.value_counts()

In [ ]:
data["method"] = data.bucket_enc.map(str) + " " + data.text_method_enc

In [ ]:
data.head()

In [ ]:
round_prec = 2

In [ ]:
tmp_auc = data[(data.metric=="auc")]

In [ ]:
tmp_auc.fillna(0, inplace=True)

In [ ]:
all_mean_aucs = tmp_auc.groupby(["dataset", "method", "cls"]).apply(lambda group: np.average(group["score"], weights=group["n_cases"])).reset_index()
all_mean_aucs.columns = ["dataset", "method", "cls", "score"]

In [ ]:
best_mean_aucs = all_mean_aucs.groupby(["dataset"])["score"].max().round(round_prec)

In [ ]:
best_mean_aucs

In [ ]:
bucket_encs = ["single_agg", "single_laststate", "prefix_index_agg", "prefix_index_last"]
text_methods = ["no text", "BoNG", "NB", "LDA", "PV"]
methods = ["%s %s" % (bucket_enc, text_method) for bucket_enc in bucket_encs for text_method in text_methods]
methods.append("prefix_index no text")
cls_methods = ["rf", "xgboost", "logit"]
datasets = ["DR", "LtC", "Github"]

In [ ]:
methods

In [ ]:
tmp2 = tmp_auc.groupby("method")

In [ ]:
print("\\toprule")
print(" & ".join([""] + ["\\multicolumn{3}{c}{%s}" % cls for cls in cls_methods]))
print("\\\\")
print(" & ".join([""] + ["%s"%dataset.replace("_", "\\_") for dataset in datasets] * len(cls_methods)))
print("\\\\ \\midrule")
for method in methods:
    elems_to_print = [method.replace("_", "\\_")]
    if method not in tmp2.groups:
        continue
    tmp_method = tmp2.get_group(method)

    tmp_method_cls = tmp_method.groupby("cls")
    for cls in cls_methods:
        tmp_cls = tmp_method_cls.get_group(cls)
        tmp_method_grouped = tmp_cls.groupby("dataset")
        for dataset_name in datasets:            
            if dataset_name not in tmp_method_grouped.groups:
                elems_to_print.append("$-$ ")
                continue
            tmp_dataset = tmp_method_grouped.get_group(dataset_name)

            mean_value = np.average(tmp_dataset["score"], weights=tmp_dataset["n_cases"])
            variance = np.average((tmp_dataset["score"]-mean_value)**2, weights=tmp_dataset["n_cases"])
            std_value = round(np.sqrt(variance), round_prec)
            mean_value = round(mean_value, round_prec)

            current_str = ""
            if mean_value == best_mean_aucs[dataset_name]:
                current_str = "$\\mathbf{%s}$ "%(mean_value)
            else:
                current_str = "$%s$ " % (mean_value)

            elems_to_print.append(current_str)

    print("%s \\\\"%(" & ".join(elems_to_print)))
        
print("\\bottomrule")

# Inter-run stability

In [183]:
data = pd.DataFrame()
for file in glob.glob("results_stability_interrun/*"):
    tmp = pd.read_csv(file, sep=";")
    if "rf" in file:
        method_name = "rf_"
    else:
        method_name = "xgboost_"
    if "rmspd5" in file:
        method_name += "auc1_rmspd5"
    else:
        method_name += "auc1_rmspd0"
    tmp["stab_method"] = method_name
    data = pd.concat([data, tmp], axis=0)

In [184]:
for file in glob.glob("results_stability/*single_agg*"):
    tmp = pd.read_csv(file, sep=";")
    if "rf" in file:
        tmp["stab_method"] = "rf_orig"
    else:
        tmp["stab_method"] = "xgboost_orig"
    data = pd.concat([data, tmp], axis=0)

In [185]:
with open("n_test_cases.pickle", "rb") as fin:
    test_cases_dict = pickle.load(fin)
    
df_test_cases = pd.DataFrame(test_cases_dict)
df_test_cases = df_test_cases.stack().reset_index()
df_test_cases.columns = ["nr_events", "dataset", "n_cases"]

data = pd.merge(data, df_test_cases, on=["dataset", "nr_events"], how="left")

In [186]:
data.head()

,dataset,method,cls,nr_events,metric,score,stab_method,n_cases
0,production,single_agg,rf_calibrated,1,auc,0.621429,rf_auc1_rmspd0,44.0
1,production,single_agg,rf_calibrated,10,auc,0.944444,rf_auc1_rmspd0,9.0
2,production,single_agg,rf_calibrated,11,auc,1.000000,rf_auc1_rmspd0,7.0
3,production,single_agg,rf_calibrated,12,auc,1.000000,rf_auc1_rmspd0,6.0
4,production,single_agg,rf_calibrated,13,auc,1.000000,rf_auc1_rmspd0,6.0


In [187]:
data.dataset = data.dataset.str.replace("insurance_activity", "insurance_1")
data.dataset = data.dataset.str.replace("traffic_fines_1", "traffic")
data.dataset = data.dataset.str.replace("insurance_followup", "insurance_2")
data.dataset = data.dataset.str.replace("bpic2017_accepted", "bpic2017_1")
data.dataset = data.dataset.str.replace("bpic2017_refused", "bpic2017_2")
data.dataset = data.dataset.str.replace("bpic2017_cancelled", "bpic2017_3")
data.dataset = data.dataset.str.replace("bpic2012_accepted", "bpic2012_1")
data.dataset = data.dataset.str.replace("bpic2012_declined", "bpic2012_2")
data.dataset = data.dataset.str.replace("bpic2012_cancelled", "bpic2012_3")
data.dataset = data.dataset.str.replace("sepsis_cases_1", "sepsis_1")
data.dataset = data.dataset.str.replace("sepsis_cases_2", "sepsis_2")
data.dataset = data.dataset.str.replace("sepsis_cases_4", "sepsis_3")
data.dataset = data.dataset.str.replace("bpic2011_f1", "bpic2011_1")
data.dataset = data.dataset.str.replace("bpic2011_f2", "bpic2011_2")
data.dataset = data.dataset.str.replace("bpic2011_f3", "bpic2011_3")
data.dataset = data.dataset.str.replace("bpic2011_f4", "bpic2011_4")
data.dataset = data.dataset.str.replace("bpic2015_1_f2", "bpic2015_1")
data.dataset = data.dataset.str.replace("bpic2015_2_f2", "bpic2015_2")
data.dataset = data.dataset.str.replace("bpic2015_3_f2", "bpic2015_3")
data.dataset = data.dataset.str.replace("bpic2015_4_f2", "bpic2015_4")
data.dataset = data.dataset.str.replace("bpic2015_5_f2", "bpic2015_5")
data.dataset = data.dataset.str.replace("hospital_billing_2", "hospital_1")
data.dataset = data.dataset.str.replace("hospital_billing_3", "hospital_2")
data.dataset = data.dataset.str.replace("dc", "DR")
data.dataset = data.dataset.str.replace("crm2", "LtC")

In [188]:
methods = ["rf_orig", "rf_auc1_rmspd0", "rf_auc1_rmspd5", "xgboost_orig", "xgboost_auc1_rmspd0", "xgboost_auc1_rmspd5"]
datasets = ["bpic2011_%s"%formula for formula in range(1,5)] + ["bpic2015_%s"%(municipality) for municipality in range(1,6)] + ["production",
                                                                                                                                    "insurance_1", "insurance_2",
                                                                                                                                    "sepsis_1", "sepsis_2", "sepsis_3",
                                                                                                                                    "bpic2012_1", "bpic2012_2", "bpic2012_3",
                                                                                                                                    "bpic2017_1", "bpic2017_2", "bpic2017_3",
                                                                                                                                    "traffic", "hospital_1", "hospital_2",
                                                                                                                               "DR", "github", "LtC"]
round_prec_auc = 3
round_prec_stab = 3

In [189]:
tmp_auc = data[(data.metric=="auc")].fillna(0)
tmp_stab = data[(data.metric=="stability")].fillna(0)

In [190]:
all_mean_aucs = tmp_auc.groupby(["dataset", "stab_method"]).apply(lambda group: np.average(group["score"], weights=group["n_cases"])).reset_index()
all_mean_aucs.columns = ["dataset", "stab_method", "score"]

all_mean_stab = tmp_stab[["dataset", "stab_method", "score"]]

In [191]:
best_mean_aucs = all_mean_aucs.groupby(["dataset"])["score"].max().round(round_prec_auc)
best_mean_stabs = all_mean_stab.groupby(["dataset"])["score"].max().round(round_prec_stab)

In [192]:
best_mean_aucs

dataset
DR             0.981
LtC            0.630
bpic2011_1     0.954
bpic2011_2     0.973
bpic2011_3     0.980
bpic2011_4     0.887
bpic2012_1     0.678
bpic2012_2     0.566
bpic2012_3     0.709
bpic2015_1     0.859
bpic2015_2     0.922
bpic2015_3     0.891
bpic2015_4     0.877
bpic2015_5     0.881
bpic2017_1     0.846
bpic2017_2     0.821
bpic2017_3     0.813
github         0.740
hospital_1     0.897
hospital_2     0.758
insurance_1    0.879
insurance_2    0.831
production     0.719
sepsis_1       0.401
sepsis_2       0.869
sepsis_3       0.714
traffic        0.661
Name: score, dtype: float64

In [193]:
best_mean_stabs

dataset
DR             0.992
LtC            0.995
bpic2011_1     0.953
bpic2011_2     0.958
bpic2011_3     0.975
bpic2011_4     0.996
bpic2012_1     0.943
bpic2012_2     0.980
bpic2012_3     0.952
bpic2015_1     0.943
bpic2015_2     0.969
bpic2015_3     0.946
bpic2015_4     0.955
bpic2015_5     0.948
bpic2017_1     0.900
bpic2017_2     0.973
bpic2017_3     0.900
github         0.922
hospital_1     0.920
hospital_2     0.987
insurance_1    0.954
insurance_2    0.935
production     0.970
sepsis_1       0.999
sepsis_2       0.994
sepsis_3       0.993
traffic        0.754
Name: score, dtype: float64

In [194]:
tmp_auc = tmp_auc.groupby("dataset")
tmp_stab = tmp_stab.groupby("dataset")

In [195]:
print("\\toprule")
print(" & \\multicolumn{6}{c}{Prediction accuracy (AUC)} & \\multicolumn{6}{c}{Temporal stability}")
print("\\\\")
print("Dataset & RF & RF\\_5 & RF\\_5\\_S & XGB & XGB\\_5 & XGB\\_5\\_S  & RF & RF\\_5 & RF\\_5\\_S & XGB & XGB\\_5 & XGB\\_5\\_S ")
print("\\\\ \\midrule")
for dataset_name in datasets:            
    elems_to_print = [dataset_name.replace("_", "\\_")]
    if dataset_name not in tmp_auc.groups:
        continue
    tmp_method = tmp_auc.get_group(dataset_name)
    tmp_method_grouped = tmp_method.groupby("stab_method")
    for method in methods:
        if method not in tmp_method_grouped.groups:
            elems_to_print.append("$-$ ")
            continue
        tmp_dataset = tmp_method_grouped.get_group(method)

        mean_value = np.average(tmp_dataset["score"], weights=tmp_dataset["n_cases"])
        mean_value = round(mean_value, round_prec_auc)

        if mean_value == best_mean_aucs[dataset_name]:
            current_str = "$\\mathbf{%s}$ "%(mean_value)
        else:
            current_str = "$%s$ " % (mean_value)

        elems_to_print.append(current_str)
        
    tmp_method = tmp_stab.get_group(dataset_name)
    tmp_method_grouped = tmp_method.groupby("stab_method")
    for method in methods:
        if method not in tmp_method_grouped.groups:
            elems_to_print.append("$-$ ")
            continue
        tmp_dataset = tmp_method_grouped.get_group(method)

        mean_value = np.average(tmp_dataset["score"])
        mean_value = round(mean_value, round_prec_stab)

        if mean_value == best_mean_stabs[dataset_name]:
            current_str = "$\\mathbf{%s}$ "%(mean_value)
        else:
            current_str = "$%s$ " % (mean_value)

        elems_to_print.append(current_str)

    print("%s \\\\"%(" & ".join(elems_to_print)))
        
print("\\bottomrule")

\toprule
 & \multicolumn{6}{c}{Prediction accuracy (AUC)} & \multicolumn{6}{c}{Temporal stability}
\\
Dataset & RF & RF\_5 & RF\_5\_S & XGB & XGB\_5 & XGB\_5\_S  & RF & RF\_5 & RF\_5\_S & XGB & XGB\_5 & XGB\_5\_S 
\\ \midrule
bpic2011\_1 & $0.937$  & $0.935$  & $0.898$  & $0.944$  & $\mathbf{0.954}$  & $0.925$  & $0.943$  & $0.947$  & $0.944$  & $0.951$  & $\mathbf{0.953}$  & $0.948$  \\
bpic2011\_2 & $0.972$  & $0.972$  & $\mathbf{0.973}$  & $0.967$  & $0.964$  & $0.969$  & $0.949$  & $0.956$  & $\mathbf{0.958}$  & $0.948$  & $0.95$  & $0.942$  \\
bpic2011\_3 & $0.979$  & $0.979$  & $0.979$  & $\mathbf{0.98}$  & $0.979$  & $\mathbf{0.98}$  & $0.972$  & $\mathbf{0.975}$  & $0.965$  & $0.971$  & $0.974$  & $0.969$  \\
bpic2011\_4 & $0.883$  & $0.871$  & $\mathbf{0.887}$  & $0.852$  & $0.865$  & $0.884$  & $0.987$  & $0.991$  & $0.987$  & $0.993$  & $\mathbf{0.996}$  & $0.991$  \\
bpic2015\_1 & $0.834$  & $0.827$  & $0.837$  & $\mathbf{0.859}$  & $0.793$  & $0.84$  & $0.934$  & $0.933$  

# Brier scores

In [21]:
data = pd.DataFrame()
for file in glob.glob("results_stability_brier/*"):
    tmp = pd.read_csv(file, sep=";")
    if "calibrated" in file:
        tmp["calibrated"] = "cal"
    else:
        tmp["calibrated"] = "uncal"
    if "prefix_index_rf" in file:
        tmp["method"] = "RF_idx_mul"
    elif "prefix_index_xgboost" in file:
        tmp["method"] = "XGB_idx_mul"
    elif "single_index_rf" in file:
        tmp["method"] = "RF_idx_pad"
    elif "single_index_xgboost" in file:
        tmp["method"] = "XGB_idx_pad"
    elif "single_agg_rf" in file:
        tmp["method"] = "RF_agg"
    elif "single_agg_xgboost" in file:
        tmp["method"] = "XGB_agg"
    elif "lstm" in file:
        tmp["method"] = "LSTM"
    data = pd.concat([data, tmp], axis=0)

In [22]:
data["cls"] = data["cls"].str.replace("_calibrated", "")

In [23]:
data.tail()

,dataset,method,cls,nr_events,metric,score,calibrated
4,dc,XGB_agg,xgboost,5,brier,0.180381,uncal
5,dc,XGB_agg,xgboost,6,brier,0.075760,uncal
6,dc,XGB_agg,xgboost,7,brier,0.888106,uncal
7,dc,XGB_agg,xgboost,8,brier,0.887131,uncal
8,dc,XGB_agg,xgboost,-1,brier,0.023363,uncal


In [24]:
data.dataset = data.dataset.str.replace("insurance_activity", "insurance_1")
data.dataset = data.dataset.str.replace("traffic_fines_1", "traffic")
data.dataset = data.dataset.str.replace("insurance_followup", "insurance_2")
data.dataset = data.dataset.str.replace("bpic2017_accepted", "bpic2017_1")
data.dataset = data.dataset.str.replace("bpic2017_refused", "bpic2017_2")
data.dataset = data.dataset.str.replace("bpic2017_cancelled", "bpic2017_3")
data.dataset = data.dataset.str.replace("bpic2012_accepted", "bpic2012_1")
data.dataset = data.dataset.str.replace("bpic2012_declined", "bpic2012_2")
data.dataset = data.dataset.str.replace("bpic2012_cancelled", "bpic2012_3")
data.dataset = data.dataset.str.replace("sepsis_cases_1", "sepsis_1")
data.dataset = data.dataset.str.replace("sepsis_cases_2", "sepsis_2")
data.dataset = data.dataset.str.replace("sepsis_cases_4", "sepsis_3")
data.dataset = data.dataset.str.replace("bpic2011_f1", "bpic2011_1")
data.dataset = data.dataset.str.replace("bpic2011_f2", "bpic2011_2")
data.dataset = data.dataset.str.replace("bpic2011_f3", "bpic2011_3")
data.dataset = data.dataset.str.replace("bpic2011_f4", "bpic2011_4")
data.dataset = data.dataset.str.replace("bpic2015_1_f2", "bpic2015_1")
data.dataset = data.dataset.str.replace("bpic2015_2_f2", "bpic2015_2")
data.dataset = data.dataset.str.replace("bpic2015_3_f2", "bpic2015_3")
data.dataset = data.dataset.str.replace("bpic2015_4_f2", "bpic2015_4")
data.dataset = data.dataset.str.replace("bpic2015_5_f2", "bpic2015_5")
data.dataset = data.dataset.str.replace("hospital_billing_2", "hospital_1")
data.dataset = data.dataset.str.replace("hospital_billing_3", "hospital_2")
data.dataset = data.dataset.str.replace("dc", "DR")
data.dataset = data.dataset.str.replace("crm2", "LtC")

In [25]:
all_briers = data[data.nr_events==-1].groupby(["dataset", "method", "calibrated"]).apply(lambda group: np.average(group["score"])).reset_index()
all_briers.columns = ["dataset", "method", "calibrated", "score"]

In [26]:
best_briers = all_briers.groupby(["dataset"])["score"].min().round(2)

In [27]:
best_briers

dataset
DR             0.01
LtC            0.11
bpic2011_1     0.11
bpic2011_2     0.05
bpic2011_3     0.03
bpic2011_4     0.13
bpic2012_1     0.21
bpic2012_2     0.15
bpic2012_3     0.16
bpic2015_1     0.09
bpic2015_2     0.05
bpic2015_3     0.07
bpic2015_4     0.06
bpic2015_5     0.08
bpic2017_1     0.12
bpic2017_2     0.07
bpic2017_3     0.15
github         0.20
hospital_1     0.03
hospital_2     0.04
insurance_1    0.09
insurance_2    0.20
production     0.22
sepsis_1       0.10
sepsis_2       0.06
sepsis_3       0.14
traffic        0.18
Name: score, dtype: float64

In [28]:
methods = ["RF_agg", "XGB_agg", "RF_idx_mul", "XGB_idx_mul", "RF_idx_pad", "XGB_idx_pad", "LSTM"]

In [29]:
datasets = ["bpic2011_%s"%formula for formula in range(1,5)] + ["bpic2015_%s"%(municipality) for municipality in range(1,6)] + ["production",
                                                                                                                                    "insurance_1", "insurance_2",
                                                                                                                                    "sepsis_1", "sepsis_2", "sepsis_3",
                                                                                                                                    "bpic2012_1", "bpic2012_2", "bpic2012_3",
                                                                                                                                    "bpic2017_1", "bpic2017_2", "bpic2017_3",
                                                                                                                                    "traffic", "hospital_1", "hospital_2",
                                                                                                                               "DR", "github", "LtC"]

In [30]:
tmp = data[data.nr_events==-1].groupby("dataset")

In [31]:
tmp.groups

{'DR': Int64Index([8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8], dtype='int64'),
 'LtC': Int64Index([20, 33, 33, 20, 33, 20, 20, 20, 20, 20, 20, 33, 33, 33], dtype='int64'),
 'bpic2011_1': Int64Index([36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36, 36], dtype='int64'),
 'bpic2011_2': Int64Index([40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40], dtype='int64'),
 'bpic2011_3': Int64Index([31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31, 31], dtype='int64'),
 'bpic2011_4': Int64Index([40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40], dtype='int64'),
 'bpic2012_1': Int64Index([40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40], dtype='int64'),
 'bpic2012_2': Int64Index([40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40], dtype='int64'),
 'bpic2012_3': Int64Index([40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40], dtype='int64'),
 'bpic2015_1': Int64Index([40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40], dtype='int64'),
 'bpic2015_2': Int64Index([40

In [32]:
print("\\toprule")
print(" & ".join([""] + ["\\multicolumn{2}{c}{%s}"%method.replace("_", "\\_") for method in methods]))
print("\\\\")
print(" & ".join([""] + ["uncal & cal"] * len(methods)))
print("\\\\ \\midrule")
for dataset in datasets:
    if dataset not in tmp.groups:
        continue
    elems_to_print = [dataset.replace("_", "\\_")]
    tmp_dataset = tmp.get_group(dataset)
    tmp_method = tmp_dataset.groupby("method")
    for method in methods:
        if method not in tmp_method.groups:
            current_str = "$-$ & $-$"
        else:
            tmp_current = tmp_method.get_group(method)
            uncal = np.round(tmp_current[tmp_current.calibrated=="uncal"].score.mean(), 2)
            cal = np.round(tmp_current[tmp_current.calibrated=="cal"].score.mean(), 2)
            if uncal == best_briers[dataset] and cal == best_briers[dataset]:
                current_str = "$\mathbf{%s}$ & $\mathbf{%s}$" % (uncal, cal)
            elif uncal == best_briers[dataset]:
                current_str = "$\mathbf{%s}$ & $%s$" % (uncal, cal)
            elif cal == best_briers[dataset]:
                current_str = "$%s$ & $\mathbf{%s}$" % (uncal, cal)
            else:
                current_str = "$%s$ & $%s$" % (uncal, cal)
            
        elems_to_print.append(current_str)

    print("%s \\\\"%(" & ".join(elems_to_print)))

print("\\bottomrule")

\toprule
 & \multicolumn{2}{c}{RF\_agg} & \multicolumn{2}{c}{XGB\_agg} & \multicolumn{2}{c}{RF\_idx\_mul} & \multicolumn{2}{c}{XGB\_idx\_mul} & \multicolumn{2}{c}{RF\_idx\_pad} & \multicolumn{2}{c}{XGB\_idx\_pad} & \multicolumn{2}{c}{LSTM}
\\
 & uncal & cal & uncal & cal & uncal & cal & uncal & cal & uncal & cal & uncal & cal & uncal & cal
\\ \midrule
bpic2011\_1 & $\mathbf{0.11}$ & $\mathbf{0.11}$ & $0.13$ & $\mathbf{0.11}$ & $\mathbf{0.11}$ & $0.12$ & $0.13$ & $0.15$ & $0.13$ & $0.15$ & $0.15$ & $0.13$ & $0.14$ & $0.16$ \\
bpic2011\_2 & $\mathbf{0.05}$ & $\mathbf{0.05}$ & $\mathbf{0.05}$ & $0.06$ & $0.12$ & $0.13$ & $0.2$ & $0.2$ & $0.12$ & $0.13$ & $0.13$ & $0.16$ & $0.14$ & $0.13$ \\
bpic2011\_3 & $0.04$ & $0.04$ & $0.12$ & $\mathbf{0.03}$ & $0.06$ & $0.07$ & $0.08$ & $0.07$ & $0.06$ & $0.07$ & $0.06$ & $0.06$ & $0.13$ & $0.14$ \\
bpic2011\_4 & $\mathbf{0.13}$ & $0.14$ & $0.18$ & $0.16$ & $0.16$ & $0.15$ & $0.17$ & $0.16$ & $0.16$ & $0.15$ & $0.21$ & $0.16$ & $0.16$ & $0.16$ \\
bpi